In [1]:
# coding: utf-8
import sys
sys.path.append('..')
from dataset import sequence


(x_train, t_train), (x_test, t_test) = \
    sequence.load_data('addition.txt', seed=1984)
char_to_id, id_to_char = sequence.get_vocab()

def to_one_hot(t_train, vocab_size):
    # 使用NumPy的eye函数创建一个独热编码矩阵
    one_hot_matrix = np.eye(vocab_size)[t_train]
    return torch.tensor(one_hot_matrix)


In [2]:
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from common.time_layers import *
from dataset import ptb

In [3]:
vocab_size = len(char_to_id) 
wordvec_size = 16
hidden_size = 128
batch_size = 128
max_epoch = 25
max_grad = 5.0

In [4]:
class Encoder(nn.Module):
    def __init__(self,vocab_size , wordvec_size , hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size,wordvec_size)
        self.lstm = nn.LSTM(wordvec_size, hidden_size, batch_first = True)
        
    def forward(self, seq, h_0, c_0):
        N , T  = seq.size()
        out = self.embed(seq)
        out , (h_0 , c_0) = self.lstm(out, (h_0 , c_0))
        
        return h_0

In [5]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size,wordvec_size)
        self.lstm = nn.LSTM(wordvec_size, hidden_size, batch_first = True)
        self.linear = nn.Linear(hidden_size,vocab_size)
        
        
    def forward(self, seq, h_0, c_0):
        N , T  = seq.size()
        out = self.embed(seq)
        out , (h_0 , c_0) = self.lstm(out, (h_0 , c_0))
        out = out.reshape(N*T,-1);
        out = self.linear(out)
        
        return out 

In [10]:
class seq2seq(nn.Module):
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.encoder = Encoder(vocab_size, wordvec_size, hidden_size)
        self.decoder = Decoder(vocab_size, wordvec_size, hidden_size)
        
        
    def forward(self, xs , decoder_xs , encoder_h_0, encoder_c_0, decoder_c_0):
        N , T  = decoder_xs.size()
        h = self.encoder(xs,encoder_h_0,encoder_c_0 )
        out = self.decoder(decoder_xs, h,decoder_c_0)
        
        return out 
        

In [7]:
mask = np.random.choice(x_train.shape[0],batch_size)

In [ ]:
test_encoder = Encoder( vocab_size, wordvec_size, hidden_size)
test_decoder = Decoder( vocab_size, wordvec_size, hidden_size)

In [ ]:
h_0 = test_encoder(seq, h_0, c_01)
c_02 = torch.zeros(1, batch_size, hidden_size)

In [11]:
test_seq2seq = seq2seq(vocab_size, wordvec_size, hidden_size)

In [22]:
seq = torch.from_numpy(x_train[mask])
encoder_h_0 = torch.zeros(1, batch_size, hidden_size)
encoder_c_0 = torch.zeros(1, batch_size, hidden_size)
tseq = torch.from_numpy(t_train[mask])[:,:-1]
decoder_c_0 = torch.zeros(1, batch_size, hidden_size)


In [31]:
tseqtrue = to_one_hot(t_train[mask][:,1:],vocab_size)
tseqtrue = tseqtrue.reshape(-1,vocab_size)

In [32]:
tseqtrue

tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [23]:
 out = test_seq2seq(seq ,tseq, encoder_h_0,encoder_c_0,decoder_c_0)

In [24]:
cri = nn.CrossEntropyLoss()

In [34]:
loss = cri(out,tseqtrue)

In [35]:
loss.backward()

In [25]:
def to_one_hot(t_train, vocab_size):
    # 使用NumPy的eye函数创建一个独热编码矩阵
    one_hot_matrix = np.eye(vocab_size)[t_train]
    return torch.tensor(one_hot_matrix)